Code:  https://github.com/Currie32/Spell-Checker/blob/master/SpellChecker.ipynb


Formato Texto: Hace referencias a los archivos de texto plano o texto enriquecido (.rtf por ejemplo)

In [ ]:
#!pip install tensorflow==1.1
#import tensorflow as tf
#print(tf.__version__)

     |████████████████████████████████| 31.4MB 141kB/s 
ERROR: stable-baselines 2.2.1 has requirement tensorflow>=1.5.0, but you'll have tensorflow 1.1.0 which is incompatible.
ERROR: magenta 0.3.19 has requirement tensorflow>=1.12.0, but you'll have tensorflow 1.1.0 which is incompatible.
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:455: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:456: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:457: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or 

1.1.0


#***Importacion de Librerias***

In [ ]:
'''
* Importacion de Librerias necesarias para la ejecucion
'''
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from collections import namedtuple
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
import time
import re
from sklearn.model_selection import train_test_split
from google.colab import drive

#***Lectura del Corpus***

In [ ]:
'''
* Monta la unidad drive
'''
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
'''
* Precondicion: Los libros deben estar en formato texto
* Input:  Directorio del libro
* Output: Texto del Libro
* Postcondicion: EL tipo de dato de book es String, lo cual debemos almacenar o utilizar ya que se reutiliza la variable
'''
def load_book(path):
    input_file = os.path.join(path)
    with open(input_file) as f:
        book = f.read()
    return book

In [ ]:
'''
* Precondicion: En el directorio debe exisitir y contener archivos en formato texto
* Input:  Directorio donde se encuentra los libros
* Output: Vector que contiene rutas de libros
* Postcondicion: Solo contiene la ruta para el uso constante de ello
Nota: Podemos reemplazar esta funcion con el ls usualmente utilizada
'''
path = '/content/gdrive/My Drive/DataSets/Corpus/'
book_files = [f for f in listdir(path) if isfile(join(path, f))]
print(book_files)
#book_files = book_files[1:]


['Corpus.txt']


**NOTA** *Probar que ocurre si hay pdf, csv, entre otros formatos, de texto como se ve que toma?*

In [ ]:
'''
* Precondicion: Debe haber almenos un libro en formato texto
* Input:  Listado de directorio previamente obtenido
* Output: Vector de libros (String que contiene los libros)
* Postcondicion: Los textos son string, se puede realizar cualquier operacion que involucre de formato string
Nota: Si usamos ls, esta funcion nos permitira ahorrar el concatenamiento de la ruta y el directorio, siendo el codigo mas legible
'''
books = []
for book in book_files:
    books.append(load_book(path+book))


In [ ]:
'''
* Precondicion: Debe haber ejecutado el proceso anterior y tener almenos un libro en el vector
* Input:  Vector de libros
* Output: Cantidad de palabras en ese libro
* Postcondicion: Ninguna
Nota: Este modulo nos ayudará a tener cierto control con la base de datos y la cantidad de palabras que estamos usando 
'''
for i in range(len(books)):
    print("Hay {} cantidad de palabras en el libro {}.".format(len(books[i].split()), book_files[i]))

Hay 16559 cantidad de palabras en el libro Corpus.txt.


In [ ]:
'''
Vemos que obtenemos como salida de un cierto libro
'''
books[0][:500]

'Me gusta el navegar en kayak.Los kiwis son una excelente fuente de vitamina C.\n\nPAPELES QUE A LA MUERTE DE GÜEMES\nQUEDARON EN PODER DE LA FAMILIA\n\nLa primera información, apoyada en documentos, con que\ncontamos sobre la suerte corrida por los "papeles de Güemes"\nnos la dan las cartas, hoy en nuestro poder, que Martín, el hijo\nprimogénito de Güemes, mientras acompañaba a algunos exilados\ntíos maternos en el Perú, escribió en distintas oportunidades\ndesde el Cerro de Pasco a su hermano Luis, de re'

# ***Preparacion del Corpus***

In [ ]:
'''
* Precondicion: Un texto netamente puro
* Input:  El texto = El libro
* Output: Libro limpio sin caracteres especiales, tabulaciones, entre otros.
* Postcondicion: Ninguna
NOTA: Podemos usar el limpiador que utiliza el Dr. Xamena
'''
def clean_text(text):
    text = re.sub(r'\n', ' ', text) 
    text = re.sub(r'[{}@_*>()\\#%+=\[\]]','', text)
    text = re.sub('a0','', text)
    text = re.sub('\'92t','\'t', text)
    text = re.sub('\'92s','\'s', text)
    text = re.sub('\'92m','\'m', text)
    text = re.sub('\'92ll','\'ll', text)
    text = re.sub('\'91','', text)
    text = re.sub('\'92','', text)
    text = re.sub('\'93','', text)
    text = re.sub('\'94','', text)
    text = re.sub('\.','. ', text)
    text = re.sub('\!','! ', text)
    text = re.sub('\?','? ', text)
    text = re.sub(' +',' ', text)
    return text

In [ ]:
'''
* Precondicion: Array de libros sin limpiar
* Input: Array de libros
* Output: Array de libros sin caracteres especiales, etc., es decir, limpios.
'''
clean_books = []
for book in books:
    clean_books.append(clean_text(book))

In [ ]:
'''
* Controlamos un texto alatorio como se ve para seguir refinando la limpieza
'''
clean_books[0][500:1500]

'reso éste en Salta desde el mismo lugar. Gracias a las del año 1847, sabemos lo siguiente: que, tras la muerte de Güemes 1821 y la de su viuda, doña Carmen Puch 1822, por minoridad de sus hijos, nacidos en 1817 y 1819 respectivamente, dichos papeles quedaron en manos de familiares cercanos; que quien después los tuvo en su poder y los ordenó fue el doctor José Redhead, el conocido médico de Güemes y de Belgrano, de gran prestigio y larga residencia en Salta “él fue el archivo de todo", según tales cartas; que, muerto el nombrado facultativo el 3 de junio de 1846, Martín Güemes y Puch, en procura de que tan valiosos documentos prestaran pronta utilidad a la Historia, pidió una y otra vez a Luis que recogiera y le remitiera "los papeles relativos a nuestro padre -decía- que debió dejar el doctor Redhead y algunas apuntaciones hechas por este sabio amigo" , para, con don Manuel Puch, fío de ambos, hacer preparar una "biografía", y, concluida que fuese ésta, el último de los nombrados los

In [ ]:
'''
* Precondicion: Contar con un texto casi puro para codificar los caracteres en enteros para la aplicacion del modelo
* Input: Texto de libros limpios
* Output: Vocabulario codificado con entero
* Postcondicion: vocabulario(diccionario) codificado en entero, sin tokens especiales
'''
vocab_to_int = {}
count = 0
for book in clean_books:
    for character in book:
        if character not in vocab_to_int:
            vocab_to_int[character] = count
            count += 1

In [ ]:
'''
* Precondicion: vocabulario(diccionario) codificado con enteros
* Input: Vocabulario codificado con enteros sin caracteres especiales 
* Output: Vocabulario codificado con enteros con caracteres especiales
* Postcondicion: Ninguna
Nota: Estos tokens especiales son especificamente para modelos seq2seq para el entrenamiento del mismo
'''
codes = ['<PAD>','<EOS>','<GO>','k']
for code in codes:
    vocab_to_int[code] = count
    count += 1

In [ ]:
'''
Muestra el vocabulario ordenado
'''
print("El vocabulario contiene {} caracteres.".format(len(vocab_to_int)))
print(sorted(vocab_to_int))

El vocabulario contiene 101 caracteres.
['\t', ' ', '!', '"', '$', "'", ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '<EOS>', '<GO>', '<PAD>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', 'ª', '«', 'º', '»', '¿', 'Á', 'É', 'Í', 'Ñ', 'Ó', 'Ú', 'Ü', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü', '—', '“', '”']


In [ ]:
'''
* Precondicion: Vocabulario(diccionario) codificado en enteros con caracteres especiales 
* Input: Vocabulario(diccionario) codificado en enteros con caracteres especiales
* Output: Vocabulario(diccionario)
* Postcondicion: Ninguna
'''
int_to_vocab = {}
for character, value in vocab_to_int.items():
    int_to_vocab[value] = character
print("El vocabulario contiene {} caracteres.".format(len(int_to_vocab)))
print(sorted(int_to_vocab))

El vocabulario contiene 101 caracteres.
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101]


In [ ]:
'''
* Precondicion: Libros limpios  
* Input: Array con libros limpios
* Output: Array de oraciones de los libros
* Postcondicion: Ninguna
Nota: A cada oracion del texto del libro lo convierte en oraciones separadas por la funcion split.
'''
sentences = []
for book in clean_books:
    for sentence in book.split('. '):
        sentences.append(sentence + '.')
print("Hay {} oraciones.".format(len(sentences)))

#Ejemplo
sentences[:2]

Hay 916 oraciones.


['Me gusta el navegar en kayak.',
 'Los kiwis son una excelente fuente de vitamina C.']

In [ ]:
'''
* Precondiciones: Array de oraciones
* Input: Array de oraciones
* Output: Array de oraciones, codificada los caracteres en enteros
* Postcondiciones: Ninguna
'''
int_sentences = []

for sentence in sentences:
    int_sentence = []
    for character in sentence:
        int_sentence.append(vocab_to_int[character])
    int_sentences.append(int_sentence)


In [ ]:
'''
* Precondiciones: oraciones codificada en enteros
* Input: Array de oraciones codificada en enteros
* Output: DataFrame de las oraciones
* Postcondiciones: Dataframe hecho en pandas para poder sacar facilmente estadisticos
'''
lengths = []
for sentence in int_sentences:
    lengths.append(len(sentence))
lengths = pd.DataFrame(lengths, columns=["counts"])
lengths.describe()

,counts
count,916.000000
mean,104.156114
std,130.711758
min,1.000000
25%,4.000000
50%,61.000000
75%,149.000000
max,1189.000000


In [ ]:
'''
* Precondicion: array de oraciones codificadas en entero 
* Input: array de oraciones codificadas en entero
* Output: cantidad de oraciones que se usaran para el entrenamiento y la prueba
* Postcondiciones: Ninguna
Nota: Filtraremos oraciones cortas o muy larga para el modelo este es la ventana que se puede modificar.
'''
max_length = 92
min_length = 10

good_sentences = []

for sentence in int_sentences:
    if len(sentence) <= max_length and len(sentence) >= min_length:
        good_sentences.append(sentence)

print("Usaremos {} para entrenar y probar nuestro modelo.".format(len(good_sentences)))

Usaremos 293 para entrenar y probar nuestro modelo.


In [ ]:
'''
Tomaremos solo el 0.15% de las oraciones para probar y el resto sera para entrenamiento
'''
training, testing = train_test_split(good_sentences, test_size = 0.15, random_state = 2)

print("Oraciones para Entrenamiento:", len(training))
print("Oraciones para prueba:", len(testing))

Oraciones para Entrenamiento: 249
Oraciones para prueba: 44


In [ ]:
'''
Ordenamos las oraciones para que el entrenamiento sea mas rapido y la ventana vaya creciendo de a poco
'''
training_sorted = []
testing_sorted = []

for i in range(min_length, max_length+1):
    for sentence in training:
        if len(sentence) == i:
            training_sorted.append(sentence)
    for sentence in testing:
        if len(sentence) == i:
            testing_sorted.append(sentence)

In [ ]:
'''
Veremos que las oraciones esten ordenando correctamente
'''
for i in range(5):
    print("Oracion:  " + str( training_sorted[i] ) + " Longitud: " + str( len(training_sorted[i]) ) )

Oracion:  [1, 8, 2, 43, 4, 1, 2, 30, 22, 14] Longitud: 10
Oracion:  [42, 2, 33, 1, 5, 6, 17, 9, 16, 5, 14] Longitud: 11
Oracion:  [42, 2, 1, 9, 2, 25, 16, 6, 16, 5, 48, 14] Longitud: 12
Oracion:  [23, 4, 20, 47, 16, 5, 2, 7, 51, 16, 5, 14] Longitud: 12
Oracion:  [23, 4, 20, 47, 16, 5, 2, 7, 51, 16, 5, 14] Longitud: 12


In [ ]:
'''
* Precondicion: oraciones y una probabilidad menor a 1 previamentes definidas
* Input: Array de oraciones codificadas y una probabilidad
* Output: Arrya de oraciones mal escritas
* Postcondiciones:  Ninguna
Crea errores en las oraciones, si la probabilidad es menor a threshold:
* si es mayor al 67% quita un caracter
  sino reordena un caracter
* con el 33% restante agrega un caracter 
'''

letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
           'n','o','p','q','r','s','t','u','v','w','x','y','z',]

def noise_maker(sentence, threshold):
    noisy_sentence = []
    i = 0
    while i < len(sentence):
        random = np.random.uniform(0,1,1)
        if random < threshold:
            noisy_sentence.append(sentence[i])
        else:
            new_random = np.random.uniform(0,1,1)
            if new_random > 0.67:
                if i == (len(sentence) - 1):
                    continue
                else:
                    noisy_sentence.append(sentence[i+1])
                    noisy_sentence.append(sentence[i])
                    i += 1
            elif new_random < 0.33:
                random_letter = np.random.choice(letters, 1)[0]
                noisy_sentence.append(vocab_to_int[random_letter])
                noisy_sentence.append(sentence[i])
            else:
                pass     
        i += 1
    return noisy_sentence

In [ ]:
'''
Muestras algunas oraciones creadas
'''
threshold = 0.9
for sentence in training_sorted[:5]:
    print(sentence)
    print(noise_maker(sentence, threshold))
    print()

[1, 8, 2, 43, 4, 1, 2, 30, 22, 14]
[1, 8, 2, 43, 4, 1, 2, 30, 22, 14]

[42, 2, 33, 1, 5, 6, 17, 9, 16, 5, 14]
[42, 2, 33, 1, 5, 6, 17, 9, 16, 5, 14]

[42, 2, 1, 9, 2, 25, 16, 6, 16, 5, 48, 14]
[42, 2, 9, 25, 2, 1, 16, 6, 16, 5, 48, 14]

[23, 4, 20, 47, 16, 5, 2, 7, 51, 16, 5, 14]
[23, 4, 20, 47, 9, 16, 5, 2, 7, 51, 16, 5, 14]

[23, 4, 20, 47, 16, 5, 2, 7, 51, 16, 5, 14]
[23, 4, 20, 47, 16, 5, 20, 2, 51, 16]



# **Construccion del Modelo**

In [ ]:
'''
Crea los marcadores de entrada del modelo
Nota en tensorFlow 2.x ya estos marcadores estan obsolotes y no se usan
'''

def model_inputs():    
    with tf.name_scope('inputs'): #
        inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    with tf.name_scope('targets'):
        targets = tf.placeholder(tf.int32, [None, None], name='targets')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    inputs_length = tf.placeholder(tf.int32, (None,), name='inputs_length')
    targets_length = tf.placeholder(tf.int32, (None,), name='targets_length')
    max_target_length = tf.reduce_max(targets_length, name='max_target_len')

    return inputs, targets, keep_prob, inputs_length, targets_length, max_target_length

In [ ]:
'''
* Precondiciones:
* Input:
* Output:
* Postcondiciones:
'''
def process_encoding_input(targets, vocab_to_int, batch_size):
    with tf.name_scope("process_encoding"):
        ending = tf.strided_slice(targets, [0, 0], [batch_size, -1], [1, 1])
        dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [ ]:
'''
* Precondiciones:
* Input:
* Output:
* Postcondiciones:
'''
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob, direction): 
    if direction == 1:
        with tf.name_scope("RNN_Encoder_Cell_1D"):
            for layer in range(num_layers):
                with tf.variable_scope('encoder_{}'.format(layer)):
                    lstm = tf.contrib.rnn.LSTMCell(rnn_size)

                    drop = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                         input_keep_prob = keep_prob)

                    enc_output, enc_state = tf.nn.dynamic_rnn(drop, 
                                                              rnn_inputs,
                                                              sequence_length,
                                                              dtype=tf.float32)

            return enc_output, enc_state
          
    if direction == 2:
        with tf.name_scope("RNN_Encoder_Cell_2D"):
            for layer in range(num_layers):
                with tf.variable_scope('encoder_{}'.format(layer)):
                    cell_fw = tf.contrib.rnn.LSTMCell(rnn_size)
                    cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                            input_keep_prob = keep_prob)

                    cell_bw = tf.contrib.rnn.LSTMCell(rnn_size)
                    cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                            input_keep_prob = keep_prob)

                    enc_output, enc_state = tf.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                            cell_bw, 
                                                                            rnn_inputs,
                                                                            sequence_length,
                                                                            dtype=tf.float32)
            enc_output = tf.concat(enc_output,2)
            return enc_output, enc_state[0]

In [ ]:
'''
* Precondiciones:
* Input:
* Output:
* Postcondiciones:
'''
def training_decoding_layer(dec_embed_input, targets_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_target_length):
    with tf.name_scope("Training_Decoder"):
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                            sequence_length=targets_length,
                                                            time_major=False)

        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                           training_helper,
                                                           initial_state,
                                                           output_layer) 

        training_logits, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                               output_time_major=False,
                                                               impute_finished=True,
                                                               maximum_iterations=max_target_length)
        return training_logits

In [ ]:
'''
* Precondiciones:
* Input:
* Output:
* Postcondiciones:
'''
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_target_length, batch_size):
    with tf.name_scope("Inference_Decoder"):
        start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')

        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                    start_tokens,
                                                                    end_token)

        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                            inference_helper,
                                                            initial_state,
                                                            output_layer)

        inference_logits, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                output_time_major=False,
                                                                impute_finished=True,
                                                                maximum_iterations=max_target_length)

        return inference_logits

In [ ]:
'''
* Precondiciones:
* Input:
* Output:
* Postcondiciones:
'''
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, inputs_length, targets_length, 
                   max_target_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers, direction):   
    with tf.name_scope("RNN_Decoder_Cell"):
        for layer in range(num_layers):
            with tf.variable_scope('decoder_{}'.format(layer)):
                lstm = tf.contrib.rnn.LSTMCell(rnn_size)
                dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                         input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  inputs_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')
    
    with tf.name_scope("Attention_Wrapper"):
        dec_cell = tf.contrib.seq2seq.DynamicAttentionWrapper(dec_cell,
                                                              attn_mech,
                                                              rnn_size)
    
    initial_state = tf.contrib.seq2seq.DynamicAttentionWrapperState(enc_state,
                                                                    _zero_state_tensors(rnn_size, 
                                                                                        batch_size, 
                                                                                        tf.float32))

    with tf.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input, 
                                                  targets_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_target_length)
    with tf.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_target_length,
                                                    batch_size)

    return training_logits, inference_logits

In [ ]:
'''
* Precondiciones:
* Input:
* Output:
* Postcondiciones:
'''
def seq2seq_model(inputs, targets, keep_prob, inputs_length, targets_length, max_target_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size, embedding_size, direction):
   
    enc_embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1, 1))
    enc_embed_input = tf.nn.embedding_lookup(enc_embeddings, inputs)
    enc_output, enc_state = encoding_layer(rnn_size, inputs_length, num_layers, 
                                           enc_embed_input, keep_prob, direction)
    
    dec_embeddings = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1, 1))
    dec_input = process_encoding_input(targets, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        dec_embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        inputs_length, 
                                                        targets_length, 
                                                        max_target_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers,
                                                        direction)
    
    return training_logits, inference_logits

In [ ]:
'''
* Precondiciones:
* Input:
* Output:
* Postcondiciones:
'''
def pad_sentence_batch(sentence_batch):
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [ ]:
'''
* Precondiciones:
* Input:
* Output:
* Postcondiciones:
'''
def get_batches(sentences, batch_size, threshold):

    for batch_i in range(0, len(sentences)//batch_size):
        start_i = batch_i * batch_size
        sentences_batch = sentences[start_i:start_i + batch_size]
        
        sentences_batch_noisy = []
        for sentence in sentences_batch:
            sentences_batch_noisy.append(noise_maker(sentence, threshold))
            
        sentences_batch_eos = []
        for sentence in sentences_batch:
            sentence.append(vocab_to_int['<EOS>'])
            sentences_batch_eos.append(sentence)
            
        pad_sentences_batch = np.array(pad_sentence_batch(sentences_batch_eos))
        pad_sentences_noisy_batch = np.array(pad_sentence_batch(sentences_batch_noisy))
        
        pad_sentences_lengths = []
        for sentence in pad_sentences_batch:
            pad_sentences_lengths.append(len(sentence))
        
        pad_sentences_noisy_lengths = []
        for sentence in pad_sentences_noisy_batch:
            pad_sentences_noisy_lengths.append(len(sentence))
        
        yield pad_sentences_noisy_batch, pad_sentences_batch, pad_sentences_noisy_lengths, pad_sentences_lengths

In [ ]:
# Parametros por default
epochs = 100
batch_size = 128
num_layers = 2
rnn_size = 512
embedding_size = 128
learning_rate = 0.0005
direction = 2
threshold = 0.95
keep_probability = 0.75

In [ ]:
'''
* Precondiciones:
* Input:
* Output:
* Postcondiciones:
'''
def build_graph(keep_prob, rnn_size, num_layers, batch_size, learning_rate, embedding_size, direction):

    tf.reset_default_graph()

    inputs, targets, keep_prob, inputs_length, targets_length, max_target_length = model_inputs()

    training_logits, inference_logits = seq2seq_model(tf.reverse(inputs, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      inputs_length,
                                                      targets_length,
                                                      max_target_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size,
                                                      embedding_size,
                                                      direction)

    training_logits = tf.identity(training_logits.rnn_output, 'logits')

    with tf.name_scope('predictions'):
        predictions = tf.identity(inference_logits.sample_id, name='predictions')
        tf.summary.histogram('predictions', predictions)
    masks = tf.sequence_mask(targets_length, max_target_length, dtype=tf.float32, name='masks')
    
    with tf.name_scope("cost"):
        cost = tf.contrib.seq2seq.sequence_loss(training_logits, 
                                                targets, 
                                                masks)
        tf.summary.scalar('cost', cost)

    with tf.name_scope("optimze"):
        optimizer = tf.train.AdamOptimizer(learning_rate)

        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

    merged = tf.summary.merge_all()    

    export_nodes = ['inputs', 'targets', 'keep_prob', 'cost', 'inputs_length', 'targets_length',
                    'predictions', 'merged', 'train_op','optimizer']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])

    return graph

# **Entrenamiento del Modelo**

In [ ]:
'''
* Precondiciones:
* Input:
* Output:
* Postcondiciones:
'''
def train(model, epochs, log_string):

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        testing_loss_summary = []

        iteration = 0
        
        display_step = 30 
        stop_early = 0 
        stop = 3 
        per_epoch = 3 
        testing_check = (len(training_sorted)//batch_size//per_epoch)-1

        print()
        print("Training Model: {}".format(log_string))

        train_writer = tf.summary.FileWriter('./logs/1/train/{}'.format(log_string), sess.graph)
        test_writer = tf.summary.FileWriter('./logs/1/test/{}'.format(log_string))
      
        for epoch_i in range(1, epochs+1): 
            batch_loss = 0
            batch_time = 0
            
            for batch_i, (input_batch, target_batch, input_length, target_length) in enumerate(
                    get_batches(training_sorted, batch_size, threshold)):
                start_time = time.time()
                print(start_time)
                summary, loss, _ = sess.run([model.merged,
                                             model.cost, 
                                             model.train_op], 
                                             {model.inputs: input_batch,
                                              model.targets: target_batch,
                                              model.inputs_length: input_length,
                                              model.targets_length: target_length,
                                              model.keep_prob: keep_probability})

                print(summary)
                batch_loss += loss
                end_time = time.time()
                batch_time += end_time - start_time
                train_writer.add_summary(summary, iteration)

                iteration += 1
                print(iteration)
                if batch_i % display_step == 0 and batch_i > 0:
                    print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                          .format(epoch_i,
                                  epochs, 
                                  batch_i, 
                                  len(training_sorted) // batch_size, 
                                  batch_loss / display_step, 
                                  batch_time))
                    batch_loss = 0
                    batch_time = 0

                if batch_i % testing_check == 0 and batch_i > 0:
                    batch_loss_testing = 0
                    batch_time_testing = 0
                    for batch_i, (input_batch, target_batch, input_length, target_length) in enumerate(
                            get_batches(testing_sorted, batch_size, threshold)):
                        start_time_testing = time.time()
                        summary, loss = sess.run([model.merged,
                                                  model.cost], 
                                                     {model.inputs: input_batch,
                                                      model.targets: target_batch,
                                                      model.inputs_length: input_length,
                                                      model.targets_length: target_length,
                                                      model.keep_prob: 1})

                        batch_loss_testing += loss
                        end_time_testing = time.time()
                        batch_time_testing += end_time_testing - start_time_testing

                        test_writer.add_summary(summary, iteration)

                    n_batches_testing = batch_i + 1
                    print('Testing Loss: {:>6.3f}, Seconds: {:>4.2f}'
                          .format(batch_loss_testing / n_batches_testing, 
                                  batch_time_testing))
                    
                    batch_time_testing = 0

                    testing_loss_summary.append(batch_loss_testing)
                    if batch_loss_testing <= min(testing_loss_summary):
                        print('New Record!') 
                        stop_early = 0
                        checkpoint = "./{}.ckpt".format(log_string)
                        saver = tf.train.Saver()
                        saver.save(sess, checkpoint)

                    else:
                        print("No Improvement.")
                        stop_early += 1
                        if stop_early == stop:
                            break

            if stop_early == stop:
                print("Stopping Training.")
                break
                

In [ ]:
'''
* Precondiciones:
* Input:
* Output:
* Postcondiciones:
'''
for keep_probability in [0.75]:
    for num_layers in [2]:
        for threshold in [0.95]:
            log_string = 'kp={},nl={},th={}'.format(keep_probability,
                                                    num_layers,
                                                    threshold) 
            print("true")
            model = build_graph(keep_probability, rnn_size, num_layers, batch_size, 
                                learning_rate, embedding_size, direction)
            print("true")
            train(model, epochs, log_string)

true
true

Training Model: kp=0.75,nl=2,th=0.95
1583102321.0712469


KeyboardInterrupt: ignored

# **Oraciones Erroneas Creadas**

In [ ]:
'''
* Precondiciones:
* Input:
* Output:
* Postcondiciones:
'''
def text_to_ints(text):
   
    text = clean_text(text)
    return [vocab_to_int[word] for word in text]

In [ ]:

text = "Spellin is difficult, whch is wyh you need to study everyday."
text = text_to_ints(text)


checkpoint = "./kp=0.75,nl=2,th=0.95.ckpt"

model = build_graph(keep_probability, rnn_size, num_layers, batch_size, learning_rate, embedding_size, direction) 

with tf.Session() as sess:
    saver = tf.train.Saver()
    saver.restore(sess, checkpoint)
    
    answer_logits = sess.run(model.predictions, {model.inputs: [text]*batch_size, 
                                                 model.inputs_length: [len(text)]*batch_size,
                                                 model.targets_length: [len(text)+1], 
                                                 model.keep_prob: [1.0]})[0]

pad = vocab_to_int["<PAD>"] 

print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format("".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format("".join([int_to_vocab[i] for i in answer_logits if i != pad])))